# vecteur gaussien et importance sampling

Ici on considère un vecteur gaussien $Z = (X,Y)$ centré de matrice de variance covariance $\Gamma$ tel que :
$$
\Gamma = \begin{pmatrix}
4 & -1 \\
-1 & 4 \\
\end{pmatrix}
$$
On souhaite calculer $ \mathbb{P} \left( Z \in \mathbb{O} \right) $ avec $ \mathbb{O} = \{ (x,y) \in \mathbb{R}^{2}, x \geq a, y \geq a \} $

Q1) Détermination naïve de la probabilité  

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [11]:
val_a = [3, 5, 7]
Mc = 200000

In [12]:
Gamma = np.array([ [4 , -1], [-1, 4] ])
A = np.linalg.cholesky(Gamma)

In [13]:
for val in val_a:    
        Z = np.random.normal(0,1, size = (Mc,2))
        for i in range(Mc):
            Z[i] = np.dot(A,Z[i])
            
        X = np.transpose(Z)[0]
        Y = np.transpose(Z)[1]
        
        p = ((X >= val)*(Y >= val)).mean()
        print("Pour a = {} et Mc = {}, on a p = {}".format(val,Mc,p))

Pour a = 3 et Mc = 200000, on a p = 0.00128
Pour a = 5 et Mc = 200000, on a p = 1.5e-05
Pour a = 7 et Mc = 200000, on a p = 0.0


On remarque bien que plus on s'éloigne (representé par la valeur de a) de la zone où est concentrée notre densité de probabilité, plus les chances de faire un "bon" tirage sont faibles donc pour un nombre simulation de MC trop faible, on trouve une proba nulle alors qu'analytiquement on sait que la proba est bien strictement positive. 

Q2) L'idée que nous allons explorer est la suivante. On va se ramener au calcul d'une autre espérance mais qui va nous "déplacer" dans une zone avec une plus grande chance de "bons" tirages tout en garantissant que nous calculons toujours la même probabilité. Cette méthode est appelé "importance sampling" ou "échantillonage préférentiel".      
On souhaite déterminer le point $(x_{0} , y_{0}) \in \mathbb{O}$ tel qu'il maximise la densité du vecteur gaussien Z.
$$
X = (x,y) \\
\Gamma^{-1} = \frac{1}{15} \begin{pmatrix}
4 & 1 \\
1 & 4 \\
\end{pmatrix} \\
\begin{align*}
    F_{Z}(X) &= \left( \frac{1}{\sqrt{2 \pi}} \right)^{2} \frac{1}{\sqrt{det(\Gamma)}} exp \left( - \frac{ X^{T} \Gamma^{-1} X }{2} \right) \\
    &= \frac{1}{2 \pi} \frac{1}{\sqrt{15}} exp \left( -\frac{1}{2*15} \left( 4x^2 + 2xy + 4y^2 \right) \right) \\
\end{align*}
$$
On cherche le point $(x_0,y_0)$ qui maximise $F_{Z}$, ce qui revient à chercher le point qui minimise $4x^2 + 2xy + 4y^2$.
$$
\begin{align*}
    4x^2 + 2xy + 4y^2 &= 4(x+y)^{2} - 6xy \\
    &= 4 (x+y)^{2} - \frac{3}{2} \left[ (x+y)^{2} - (x-y)^{2} \right] \\
    &= \frac{5}{2} (x+y)^{2} + \frac{3}{2} (x-y)^{2}
\end{align*}
$$
On a une somme de carrés donc tout est positif. Le point de $\mathbb{O}$ qui minimise cela est $(a,a)$. Donc le point qui maximise $F_{Z}$ est $(x_{0},y_{0})^{T} = (a,a)^{T} = m $

On considère le vecteur $W$ tel que $W \sim \mathcal{N}( m, \Gamma )$

$$ 
\begin{align*}
    p &= \mathbb{P} \left( Z \in \mathbb{O} \right) \\
    &= \mathbb{P} \left( (X,Y) \in \mathbb{O} \right) \\
    &= \mathbb{E} \left[ 1_{ \{ (X,Y) \in \mathbb{O} \} } \right] \\
    &= \int_{\mathbb{R}^{2}} 1_{ \{ (x,y) \in \mathbb{O} \} } f_{Z}(x,y) \mathrm{d}x \mathrm{d}y \\
    &= \int_{\mathbb{R}^{2}} 1_{ \{ (x,y) \in \mathbb{O} \} } \frac{f_{Z}(x,y)}{f_{W}(x,y)} f_{W}(x,y) \mathrm{d}x \mathrm{d}y \\
    &= \int_{\mathbb{R}^{2}} 1_{ \{ (x,y) \in \mathbb{O} \} } g(x,y) f_{W}(x,y) \mathrm{d}x \mathrm{d}y \\
    &=  \mathbb{E} \left[ 1_{ \{ W \in \mathbb{O} \} } g(W) \right] \\
\end{align*}
$$

$$
\begin{align*}
    g(X) &= \frac{F_Z (X)}{F_W (X)} \\
    &= \left( \frac{\sqrt{2 \pi}}{\sqrt{2 \pi}} \right)^{2} \left( \frac{\sqrt{det(\Gamma)}}{\sqrt{det(\Gamma)}} \right) \frac{ exp \left( - \frac{1}{2} \left( X^{T} \Gamma^{-1} X \right) \right) } {exp \left( - \frac{1}{2} \left( \left( X^{T} - m \right) \Gamma^{-1} \left( X - m \right) \right) \right) } \\
    &= \frac{ exp \left( - \frac{1}{2} \left( X^{T} \Gamma^{-1} X \right) \right) } {exp \left( - \frac{1}{2} \left( \left( X^{T} - m \right) \Gamma^{-1} \left( X - m \right) \right) \right) } \\
    &=  exp \left( - \frac{1}{2} \left( X^{T} \Gamma^{-1} X \right) + \frac{1}{2} \left( \left( X^{T} - m \right) \Gamma^{-1} \left( X - m \right) \right) \right) \\
    &= exp \left( - \frac{1}{2} \left(  X^{T} \Gamma^{-1} X  - X^{T} \Gamma^{-1} X + X^{T} \Gamma^{-1} m + m^{T} \Gamma^{-1} X - m^{T} \Gamma^{-1} m  \right) \right) \\
    &= exp \left( - \frac{1}{2} \left( + X^{T} \Gamma^{-1} m + m^{T} \Gamma^{-1} X - m^{T} \Gamma^{-1} m  \right) \right) \\
\end{align*}
$$

Calculons : 
$$
\begin{align*}
    X^{T} \Gamma^{-1} m &= \frac{a}{3} \left( x + y \right) \\
    X^{T} \Gamma^{-1} m &= (X^{T} \Gamma^{-1} m)^{T} = \frac{a}{3} \left( x + y \right) \\
    m^{T} \Gamma^{-1} m &= \frac{a^{2}}{3}
\end{align*}
$$

Donc : 
$$ 
\begin{equation*}
    g(X) = exp \left( - \frac{a}{6} \left( 2x + 2y -a \right) \right)
\end{equation*}
$$

In [14]:
 g = lambda x,y,a: np.exp( -(a/6)*(2*x +2*y - a)   ) 

In [17]:
print("Pour MC = {}, on a : ".format(Mc))
for val in val_a:
    W = np.random.normal(0,1, size = (Mc,2))
    for i in range(Mc):
        W[i] = np.dot(A,W[i]) + val*np.ones(2)
        
    W1 = W[:,0]
    W2 = W[:,1]
    
    a = val * np.ones(Mc)
    p = ( (W1 >= val)*(W2 >= val)*g(W1,W2,a) ).mean()
    print("Avec a = {} : p = {}".format(val,p))

Pour MC = 200000, on a : 
Avec a = 3 : p = 0.000309231202780361
Avec a = 5 : p = 4.634278036904985e-08
Avec a = 7 : p = 1.5755191674640343e-13


Q4) Ce que l'on a fais plutôt correspondait à un déplacement mais uniquement au sens d'une translation. On peut se demander ce qu'il se passe si en plus d'une translation, on ajoute une dilatation pour notre "importance sampling". 

On considère le vecteur $W$ tel que $W \sim \mathcal{N}( m, \delta \Gamma )$

$$
\begin{align*}
    g_{\delta}(X) &= \frac{F_Z (X)}{F_W (X)} \\
    &= \left( \frac{\sqrt{2 \pi}}{\sqrt{2 \pi}} \right)^{2} \left( \frac{\sqrt{det(\Gamma)}}{\sqrt{det(\delta \Gamma)}} \right) \frac{ exp \left( - \frac{1}{2} \left( X^{T} \Gamma^{-1} X \right) \right) } {exp \left( - \frac{1}{2} \left( \left( X - m \right)^{T} (\delta \Gamma)^{-1} \left( X - m \right) \right) \right) } \\
    &= \frac{1}{\delta} \frac{ exp \left( - \frac{1}{2} \left( X^{T} \Gamma^{-1} X \right) \right) } {exp \left( - \frac{1}{2} \left( \left( X - m \right)^{T} (\delta \Gamma)^{-1} \left( X - m \right) \right) \right) } \\
    &= \frac{1}{\delta} exp \left( - \frac{1}{2} \left( X^{T} \Gamma^{-1} X \right) + \frac{1}{2} \left( \left( X - m \right)^{T} (\delta \Gamma)^{-1} \left( X - m \right) \right) \right) \\
    &= \frac{1}{\delta} exp \left( - \frac{1}{2} \left( X^{T} \Gamma^{-1} X  - \left( X - m \right)^{T} (\delta \Gamma)^{-1} \left( X - m \right) \right) \right) \\
    &= \frac{1}{\delta} exp \left( - \frac{1}{2} \left( X^{T} \Gamma^{-1} X  - \frac{1}{\delta} \left( X - m \right)^{T}  \Gamma^{-1} \left( X - m \right) \right) \right) \\
    &= \frac{1}{\delta} exp \left( - \frac{1}{2} \left( \frac{2}{15} (2x^2 + xy + 2y^2)  - \frac{1}{\delta} \left( \frac{2}{15} (2x^2 + xy + 2y^2) -a \frac{2}{3} (x+y) + \frac{a^{2}}{3}  \right)  \right) \right) \\
\end{align*}
$$

In [18]:
g_delta = lambda x,y,a,delta : 1/delta * np.exp( -(1/2) * (2/15 * (2*x*x + x*y + 2*y*y) - (1/delta * ( 2/15 * (2*x*x + x*y + 2*y*y) - a*2/3 *(x+y) + a*a/3 ))  )  )

In [19]:
val_delta = np.array([0.1,0.5,1,1.5,2,2.5,3])

for val_d in val_delta:
    print("delta = {}".format(val_d))
    A_delta = np.sqrt(val_d)*A
    for val in val_a:
        W = np.random.normal(0,1, size = (Mc,2))
        for i in range(Mc):
            W[i] = np.dot(A_delta,W[i]) + val*np.ones(2)
    
        W1 = W[:,0]
        W2 = W[:,1]
    
        a = val * np.ones(Mc)
        delta = val_d * np.ones(Mc)
        p = ( (W1 >= val)*(W2 >= val)*g_delta(W1,W2,a,delta) ).mean()
        print("Pour a = {} et Mc = {}, on a p = {}".format(val,Mc,p))
    print("\n")

delta = 0.1
Pour a = 3 et Mc = 200000, on a p = 3.9848082977135024e-08
Pour a = 5 et Mc = 200000, on a p = 2.370849046500004e-22
Pour a = 7 et Mc = 200000, on a p = 1.8775019010518257e-43


delta = 0.5
Pour a = 3 et Mc = 200000, on a p = 0.00027680637279295584
Pour a = 5 et Mc = 200000, on a p = 2.8843208921634833e-09
Pour a = 7 et Mc = 200000, on a p = 1.7440984450768442e-16


delta = 1.0
Pour a = 3 et Mc = 200000, on a p = 0.0003092600441078786
Pour a = 5 et Mc = 200000, on a p = 4.588956962811125e-08
Pour a = 7 et Mc = 200000, on a p = 1.563273818258357e-13


delta = 1.5
Pour a = 3 et Mc = 200000, on a p = 0.00022775569497830977
Pour a = 5 et Mc = 200000, on a p = 8.35813660782839e-08
Pour a = 7 et Mc = 200000, on a p = 1.0493492263079104e-12


delta = 2.0
Pour a = 3 et Mc = 200000, on a p = 0.0001615848114002442
Pour a = 5 et Mc = 200000, on a p = 9.188126573612293e-08
Pour a = 7 et Mc = 200000, on a p = 2.3545989019034842e-12


delta = 2.5
Pour a = 3 et Mc = 200000, on a p = 0.000